In [185]:
!pip install selenium
!pip install beautifulsoup4
# !pip install webdriver-manager
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (169 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [186]:
import sys, time, re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller

chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"
sys.path.insert(0,chrome_path)
chromedriver_autoinstaller.install()

chrome_options = Options()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# chrome_service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
processed_links = set()

In [187]:
def parse_relative_time(post_time):
  """상대적인 시간을 파싱하고 현재 시간으로부터의 시간 차이를 계산"""
  now = datetime.now()

  try:
    time_match = re.match(r'(\d+)([초|분|시간|일|주]) 전', post_time)
  except ValueError:
    print(f"날짜 형식 변환 실패: {post_time}")
    return None

  if time_match:
    quantity = int(time_match.group(1))
    unit = time_match.group(2)

    # 시간 단위에 따라 처리
    if unit == '분':
      return now - timedelta(minutes=quantity)
    elif unit == '초':
      return now - timedelta(seconds=quantity)
    elif unit == '시간':
      # print(now - timedelta(hours=quantity))
      return now - timedelta(hours=quantity)
    elif unit == '일':
      # print(now - timedelta(days=quantity))
      return now - timedelta(days=quantity)
    elif unit == '주':
      # print(now - timedelta(weeks=quantity))
      return now - timedelta(weeks=quantity)
    else:
      raise ValueError("지원되지 않는 시간 단위입니다.")

In [188]:
def extract_blog_posts(limit, html, processed_links):
  soup = BeautifulSoup(html, 'html.parser')
  urls = []
  now = datetime.now()

  for post in soup.select('div.view_wrap'):
      title_tag = post.select_one('a.title_link')
      if title_tag:
          title = title_tag.get_text()
          href = title_tag['href']

          # 중복 확인
          if href in processed_links:
              continue
          processed_links.add(href)

          # 작성 시간 추출
          time_tag = post.select_one('span.sub')
          post_time = time_tag.get_text() if time_tag else "작성 시간 없음"

          post_datetime = parse_relative_time(post_time)

          if post_datetime:
            if now - post_datetime <= timedelta(hours=limit):
              if "blog.naver.com" in href:
                urls.append(href)
                print(f"제목: {title}, 링크: {href}, 작성 시간: {post_time}")
                print('-' * 150)
  return urls

In [189]:
def scroll_to_bottom(limit, driver):
  last_height = driver.execute_script("return document.body.scrollHeight")
  urls = []
  while True:
      print("Scrolling...")
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(2)

      new_height = driver.execute_script("return document.body.scrollHeight")
      urls.extend(extract_blog_posts(limit, driver.page_source, processed_links))

      if new_height == last_height:
          break
      last_height = new_height
  return urls

In [190]:
keywords = ['롯데카드', '로카', '롯카', 'LOCA']
limit = 1
table_related = {}
table_latest = {}

for keyword in keywords:
  related = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={keyword}&nso=so%3Ar%2Cp%3A1d"
  latest = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={keyword}&nso=so%3Add%2Cp%3A{limit}h"
  driver = webdriver.Chrome(options=chrome_options)
  print(related)
  driver.get(related)
  driver.implicitly_wait(3)
  print(f"{driver.title} 관련도순")
  table_related[keyword] = scroll_to_bottom(limit, driver)

  print(latest)
  driver.get(latest)
  driver.implicitly_wait(3)
  print(f"{driver.title} 최신순")
  table_latest[keyword] = scroll_to_bottom(limit, driver)

#  if "so%3Ar" in url:
#    print(f"{driver.title} 관련도순")
#    table_related[keyword] = scroll_to_bottom(limit, driver)

#  elif "so%3Add" in url:
#    print(f"{driver.title} 최신순")
#    table_latest[keyword] = scroll_to_bottom(limit, driver)

print(f"관련도순 테이블: {table_related}")
print(f"최신순 테이블: {table_latest}")

https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=롯데카드&nso=so%3Ar%2Cp%3A1d
롯데카드 : 네이버 블로그검색 관련도순
Scrolling...
제목: 롯데카드 정지 해결하고 싶다면, 링크: https://blog.naver.com/garages_/223596868876, 작성 시간: 17분 전
------------------------------------------------------------------------------------------------------------------------------------------------------
제목: 연회비 없는 신용카드 롯데카드 캐시백(아파트관리비 주유 배달앱 백화점), 링크: https://blog.naver.com/doomaker/223597222726, 작성 시간: 15분 전
------------------------------------------------------------------------------------------------------------------------------------------------------
제목: 롯데카드 스카이패스 혜택 및 특징 한눈에 정리, 링크: https://blog.naver.com/chargef2374/223597214030, 작성 시간: 21분 전
------------------------------------------------------------------------------------------------------------------------------------------------------
Scrolling...
제목: 기저귀 바우처 카드 신청 사용처 쿠팡 잔액 조회 방법, 링크: https://blog.naver.com/tlsrb2008/223597217696, 작성 시간: 18분 전
-------------

In [193]:
index = 1
for idx, keyword in enumerate(keywords, start=1):
  for idx, url in enumerate(table_related[keyword], start=1):
    # filename = f"{keyword}_{idx}.txt"
    filename = f"related_{index}.txt"
    index += 1
    with open(filename, 'w', encoding='utf-8') as file:
      driver.get(url)
      time.sleep(3)  # 필요한 경우 대기 시간을 조정
      driver.switch_to.frame('mainFrame')
      html = driver.page_source
      soup = BeautifulSoup(html, 'html.parser')

      print(f"링크: {url}")
      title_element = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text.se-title-text')
      title_text = title_element.text
      print(f"제목: {title_text}")

      publish_date_element = driver.find_element(By.CSS_SELECTOR, 'span.se_publishDate.pcol2')
      publish_date = publish_date_element.text
      post_datetime = parse_relative_time(publish_date)
      print(f"작성 날짜: {post_datetime}")

      main_content = soup.select('div.se-main-container div.se-component-content')
      content_text = "\n".join([content.get_text(strip=True) for content in main_content])
      # print(f"본문: {content_text}")

      like_count_tag = soup.select_one('em.u_cnt._count')
      like_count = like_count_tag.get_text().strip() if like_count_tag else 0

      # 댓글 수 추출 (em#commentCount._commentCount)
      comment_count_tag = soup.select_one('em#commentCount._commentCount')
      # comment_count = comment_count_tag.get_text().strip() if comment_count_tag else 0
      if comment_count_tag:
        comment_count = comment_count_tag.get_text().strip()
        comment_count = int(comment_count) if comment_count.isdigit() else 0
      else:
        comment_count = 0

      # 결과 출력
      print(f"공감 수: {like_count}")
      print(f"댓글 수: {comment_count}")

      file.write(f"키워드: {keyword}\n")
      file.write(f"링크: {url}\n")
      file.write(f"제목: {title_text}\n")
      file.write(f"작성일: {post_datetime}\n")
      file.write("출처: 네이버 블로그\n")
      file.write(f"본문: {content_text}\n")
      file.write(f"공감 수: {like_count}\n")
      file.write(f"댓글 수: {comment_count}\n")

      if comment_count > 0:
        try:
          comment_button = driver.find_element(By.CSS_SELECTOR, 'a.btn_comment')

          driver.execute_script("arguments[0].scrollIntoView(true);", comment_button)
          time.sleep(1)  # 스크롤 후 잠시 대기

          driver.execute_script("arguments[0].click();", comment_button)
          time.sleep(2)
          # print("댓글창을 열었습니다.")

          html = driver.page_source
          soup = BeautifulSoup(html, 'html.parser')
          comment_list = soup.select('div.u_cbox_text_wrap span.u_cbox_contents')
          for idx, comment in enumerate(comment_list, start=1):
            comment_text = comment.get_text(strip=True)
            print(f"댓글 {idx}: {comment_text}")
            file.write(f"댓글 {idx}: {comment_text}\n")

        except Exception as e:
          print(f"댓글 버튼을 찾을 수 없습니다: {e}")


링크: https://blog.naver.com/garages_/223596868876
제목: 롯데카드 정지 해결하고 싶다면
작성 날짜: 2024-09-26 05:20:12.526219
공감 수: 6
댓글 수: 0
링크: https://blog.naver.com/doomaker/223597222726
제목: 연회비 없는 신용카드 롯데카드 캐시백(아파트관리비 주유 배달앱 백화점)
작성 날짜: 2024-09-26 05:23:17.787439
공감 수: 9
댓글 수: 0
링크: https://blog.naver.com/chargef2374/223597214030
제목: 롯데카드 스카이패스 혜택 및 특징 한눈에 정리
작성 날짜: 2024-09-26 05:16:23.292688
공감 수: 0
댓글 수: 0
링크: https://blog.naver.com/tlsrb2008/223597217696
제목: 기저귀 바우처 카드 신청 사용처 쿠팡 잔액 조회 방법
작성 날짜: 2024-09-26 05:19:29.783466
공감 수: 11
댓글 수: 3
댓글 1: 오후는  맛점후 산책하기 좋은 날이네요오후도 화이팅입니다
댓글 2: 오와 기저귀 바우처 이런게 다 있네요!모아두었다가 세일할 때 한번에 쫙~ 사놓으면 좋겠어요! ㅎㅎ
댓글 3: 와 이거 진짜 꿀정보네요. 쿠팡도 걍 다 되는게아니고... 로켓만 되고 그 판매자 직접배송은 안되고 그렇군요 이거는 진짜 꿀정보네요 모르시는분들 많을거같아요!!
링크: https://blog.naver.com/kimsong8106/223597211268
제목: 신용카드 해지 어카운트인포에서 한 번에 가능하다는데
작성 날짜: 2024-09-26 05:14:37.702695
공감 수: 9
댓글 수: 1
댓글 1: 신용카드 해지 관련 유용한 정보 잘봤습니다.
링크: https://blog.naver.com/jinacademy_/223597196010
제목: 진단과학원 비대면 결제 방법 안내드립니다.
작성 날짜: 2024-09-26 05:02:47.88

In [194]:
index = 1
for idx, keyword in enumerate(keywords, start=1):
  for idx, url in enumerate(table_latest[keyword], start=1):
    # filename = f"{keyword}_{idx}.txt"
    filename = f"latest_{index}.txt"
    index += 1
    with open(filename, 'w', encoding='utf-8') as file:
      driver.get(url)
      time.sleep(3)  # 필요한 경우 대기 시간을 조정
      driver.switch_to.frame('mainFrame')
      html = driver.page_source
      soup = BeautifulSoup(html, 'html.parser')

      print(f"링크: {url}")
      title_element = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text.se-title-text')
      title_text = title_element.text
      print(f"제목: {title_text}")

      publish_date_element = driver.find_element(By.CSS_SELECTOR, 'span.se_publishDate.pcol2')
      publish_date = publish_date_element.text
      post_datetime = parse_relative_time(publish_date)
      print(f"작성 날짜: {post_datetime}")

      main_content = soup.select('div.se-main-container div.se-component-content')
      content_text = "\n".join([content.get_text(strip=True) for content in main_content])
      # print(f"본문: {content_text}")

      like_count_tag = soup.select_one('em.u_cnt._count')
      like_count = like_count_tag.get_text().strip() if like_count_tag else 0

      # 댓글 수 추출 (em#commentCount._commentCount)
      comment_count_tag = soup.select_one('em#commentCount._commentCount')
      # comment_count = comment_count_tag.get_text().strip() if comment_count_tag else 0
      if comment_count_tag:
        comment_count = comment_count_tag.get_text().strip()
        comment_count = int(comment_count) if comment_count.isdigit() else 0
      else:
        comment_count = 0

      # 결과 출력
      print(f"공감 수: {like_count}")
      print(f"댓글 수: {comment_count}")

      file.write(f"키워드: {keyword}\n")
      file.write(f"링크: {url}\n")
      file.write(f"제목: {title_text}\n")
      file.write(f"작성일: {post_datetime}\n")
      file.write("출처: 네이버 블로그\n")
      file.write(f"본문: {content_text}\n")
      file.write(f"공감 수: {like_count}\n")
      file.write(f"댓글 수: {comment_count}\n")

      if comment_count > 0:
        try:
          comment_button = driver.find_element(By.CSS_SELECTOR, 'a.btn_comment')

          driver.execute_script("arguments[0].scrollIntoView(true);", comment_button)
          time.sleep(1)

          driver.execute_script("arguments[0].click();", comment_button)
          time.sleep(2)
          # print("댓글창을 열었습니다.")

          html = driver.page_source
          soup = BeautifulSoup(html, 'html.parser')
          comment_list = soup.select('div.u_cbox_text_wrap span.u_cbox_contents')
          for idx, comment in enumerate(comment_list, start=1):
            comment_text = comment.get_text(strip=True)
            print(f"댓글 {idx}: {comment_text}")
            file.write(f"댓글 {idx}: {comment_text}\n")

        except Exception as e:
          print(f"댓글 버튼을 찾을 수 없습니다: {e}")

driver.quit()

링크: https://blog.naver.com/dailystore_me/223592660791
제목: 24년 9월 알리익스프레스 할인프로모션 총정리
작성 날짜: 2024-09-26 05:30:18.385053
공감 수: 0
댓글 수: 0


In [ ]:
url = 'https://blog.naver.com/kiara74/223594405525'

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(3)

driver.switch_to.frame('mainFrame')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

source = f"네이버 블로그: {url}"
print(source)

title_element = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text.se-title-text')
title_text = title_element.text
print(f"제목: {title_text}")

publish_date_element = driver.find_element(By.CSS_SELECTOR, 'span.se_publishDate.pcol2')
publish_date = publish_date_element.text
print(f"작성 날짜: {publish_date}")

main_content = soup.select('div.se-main-container div.se-component-content')
content_text = "\n".join([content.get_text(strip=True) for content in main_content])
print("본문 내용:")
print(content_text)

like_count_tag = soup.select_one('div.post-btn.post_btn2 em.u_cnt._count')
like_count = like_count_tag.get_text().strip() if like_count_tag else "공감 없음"

comment_count_tag = soup.select_one('em#commentCount._commentCount')
comment_count = comment_count_tag.get_text().strip() if comment_count_tag else "댓글 없음"

# 결과 출력
print(f"공감 수: {like_count}")
print(f"댓글 수: {comment_count}")

if comment_count != "댓글 없음":
  try:
    comment_button = driver.find_element(By.CSS_SELECTOR, 'a.btn_comment')

    driver.execute_script("arguments[0].scrollIntoView(true);", comment_button)
    time.sleep(1)  # 스크롤 후 잠시 대기

    driver.execute_script("arguments[0].click();", comment_button)
    time.sleep(2)
    print("댓글창을 열었습니다.")
    # comment_button.click()
    # time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_list = soup.select('div.u_cbox_text_wrap span.u_cbox_contents')
    for idx, comment in enumerate(comment_list, start=1):
      comment_text = comment.get_text(strip=True)
      print(f"댓글 {idx}: {comment_text}")

  except Exception as e:
    print(f"댓글 버튼을 찾을 수 없습니다: {e}")
    driver.quit()
